### Tensorflow Variable
---

1. 变量的作用和处理手段

    1. 变量用来保存训练中的参数信息，张量存放在内存的缓冲区中，可以随时的被加载到磁盘上用来做之后的分析使用
    2. 主要的类
        
        1. `tf.Variable`
        2. `tf.train.Saver`
        
2. 创建变量
3. 初始化变量
    
    1. 直接初始化
        
        1. 全部初始化
        2. 部分初始化
    2. 使用变量初始化(**`dtype`必须符合**)
    
4. 保存变量
    
   1. 最简单的保存和恢复模型的方法是使用 `tf.train.Saver` 对象。构造器给 `graph` 的所有变量，或是定义在列表里的变量，添加 `save` 和 `restoreops`。`saver` 对象提供了方法来运行这些 `ops` ，定义检查点文件的读写路径。
   2. 变量存储在二进制文件里，主要包含从变量名到 `tensor`值的映射关系。当你创建一个 `Saver` 对象时，你可以选择性地为检查点文件中的变量挑选变量名。默认情况下，将每个变量 `Variable.name` 属性的值。
   3. 如果你不给 `tf.train.Saver()` 传入任何参数，那么 `saver` 将处理 `graph` 中的所有变量。其中每一个变量都以变量创建时传入的名称被保存。
   4. 你可以通过给tf.train.Saver()构造函数传入Python字典，很容易地定义需要保持的变量及对应名称：键对应使用的名称，值对应被管理的变量。在载入的程序中，**变量的 `name` 属性**应该和字典中的键一致，如下面的程序所示

In [12]:
# 创建变量
vara = tf.Variable(1, name='var_a')

# 初始化变量
init = tf.global_variables_initializer()

# 使用变量初始化变量
varb = tf.Variable(vara.initialized_value() * 2, name="var_b")

# 部分初始化变量
tf.variables_initializer([vara])

<tf.Operation 'init_12' type=NoOp>

In [2]:
# 保存变量
v1 = tf.Variable(1, name="v1")
v2 = tf.Variable(2, name='v2')

init = tf.global_variables_initializer()

# 指定要保存的变量字典
saver = tf.train.Saver({'v3': v1, 'v4':v2})

with tf.Session() as sess:
    sess.run(init)
    save_path = saver.save(sess, './tmp/test.ckpt')
    print('Model save into:', save_path)
    print(sess.run([v1, v2]))

Model save into: ./tmp/test.ckpt
[1, 2]


In [11]:
# 加载变量, 请不要在 Jupyter 中执行，详细请参考同目录下的 test.py 脚本
with tf.Session() as sess:
    v3 = tf.Variable(30, name='v3')
    v4 = tf.Variable(40, name='v4')
    init = tf.global_variables_initializer()
    sess.run(init)
    saver = tf.train.Saver()
    saver.restore(sess, './tmp/test.ckpt')
    print(sess.run([v3, v4]))

"\nwith tf.Session() as sess:\n    v3 = tf.Variable(30, name='v3')\n    v4 = tf.Variable(40, name='v4')\n    init = tf.global_variables_initializer()\n    sess.run(init)\n    saver = tf.train.Saver()\n    saver.restore(sess, './tmp/test.ckpt')\n    print(sess.run([v3, v4]))\n    "

### Tensorflow Tensorboard
---

1. `Summary` 写入
    1. `tf.train.FileWriter`
    2. `tf.summary.Scalar`
    3. `tf.summary.merge_all`
    4. `tf.summary.histogram` : https://www.2cto.com/net/201712/706932.html
    5. `tf.train.FileWriter.add_sumaary(summary, global_step)`
   
2. 可视化

    1. 依赖关系
        
        1. 数据依赖  
           节点存在明显的 `tensor` 流动关系，在 `tensorboard` 中使用实心线连接
           
        2. 控制依赖  
           控制型，有特殊功能的节点的作用
           
           

In [1]:
# histogram 图表显示
# k 是正态分布的中值系数
k = tf.placeholder(tf.float32)

# 生成一个正态分布, 并且改变正态分布的中值mean
mean_moving_normal = tf.random_normal(shape=[1000], mean=(5*k), stddev=1)
# 记录这个分布为直方图摘要
tf.summary.histogram("moving_mean", mean_moving_normal)

# 设置一个session，并且写出summary 的events files
sess = tf.Session()
writer = tf.summary.FileWriter("/tmp/histogram_example")

summaries = tf.summary.merge_all()

# 设置一个400循环，并将这些summary写入硬盘
N = 400
for step in range(N):
    k_val = step / float(N)
    # 写入400次summary
    summ = sess.run(summaries, feed_dict={k: k_val})
    writer.add_summary(summ, global_step=step)

    writer.flush()

### Tensorflow Read data
---
1. 数据读取的方式
    
    1. `feed_dict`  
        Python代码在程序的运行的任一阶段中，将数据通过 `feed_dict` 的方式供给 `tensorflow` 的程序
    2. 文件读取
    3. 预加载在代码中
    
2. 文件读取  

    **使用文件读取管线接在 `tensorflow` 计算图中实现数据供给**
    1. 参考资料 : http://blog.csdn.net/buptgshengod/article/details/72956846
    2. 注意点
        1. 一定使用 `tf.train.Coordinator` 线程协调器
        2. 特别要强调的是， `string_input_producer` 内部创建了一个 `epoch` 计数变量，归入 `tf.GraphKeys.LOCAL_VARIABLES` 集合中，必须单独用 `global_variables_initializer` 初始化
        3. 队列读取完毕，会抛出一个异常 `tf.errors.OutOfRangeError`,提示队列已经读取完毕，这里需要用到异常处理
        
        4. 存在两个队列
            
            1. 文件名队列
            2. 样本队列 : 我们的训练循环会使得样本队列中的样本不断地出队。
        
    3. 函数讲解
        
        1. `tf.train.string_input_producer` : 来生成一个先入先出的**文件名队列**， 文件阅读器会需要它来读取数据。
        2. `tf.start_queue_runners` : 
            
            在你运行任何训练步骤之前，需要调用tf.train.start_queue_runners函数，否则数据流图将一直挂起,这个函数将会启动输入管道的线程，填充样本到队列中，以便出队操作可以从队列中拿到样本
            
    4. 难点 : https://www.jianshu.com/p/d063804fb272
        
        1. `QueueRunner` : http://blog.csdn.net/shenxiaolu1984/article/details/53024513
            
            1. Tensorflow的计算主要在使用CPU/GPU和内存，而数据读取涉及磁盘操作，速度远低于前者操作。因此通常会使用多个线程读取数据，然后使用一个线程消费数据。QueueRunner就是来管理这些读写队列的线程的。
            2. `Coordinator` 类可以用来同时停止多个工作线程并且向那个在等待所有工作线程终止的程序报告异常
            3. **是TF中对操作Queue的线程的封装**
            4. 使用 `QueueRunner` 可以创建一系列新的线程进行入队操作，让主线程继续使用数据。
        
        2. `Coordinator` : 线程管理器
        
            1. `Coordinator.request_stop` : 发生了一些事情(比如队列空异常)，请求线程集体停止
            2. `Coordinator.should_stop` : 判断是否发生了异常，是否发出了请求停止的信号，如果一个请求停止(True)，所有线程集体停止
            3. `Coordinator.join(<list of threads>)` : 等待指定的线程终止

In [ ]:
# 文件读取模板
# Create the graph, etc.
init_op = tf.local_variables_initializer()

# Create a session for running operations in the Graph.
sess = tf.Session()

# Create queue and read
filename = ['1.jpg', '2.jpg', '3.jpg']
filename_queue = tf.train.string_input_producer(filename, shuffle=False, num_epochs=5)
reader = tf.WholeFileReader()
key, value = reader.read(filename_queue)

# Initialize the variables (like the epoch counter).
sess.run(init_op)

# Start input enqueue threads.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

try:
    while not coord.should_stop():
        # Run training steps or whatever
        # The training operation
        sess.run(train_op)

except tf.errors.OutOfRangeError:
    print('Done training -- epoch limit reached')
finally:
    # When done, ask the threads to stop.
    coord.request_stop()

# Wait for threads to finish.
coord.join(threads)
sess.close()

### Tensorflow Queue and Thread
---
1. `Queue`
    
    1. `Queue` 在 `tensorflow` 中是一种队列模拟器，用来实现一些数据读取等操作　：
        
        队列本身也是图中的一个节点。其他节点（enqueue, dequeue）可以修改队列节点中的内容
        
        
    2. 所有队列管理器被默认加入图的 `tf.GraphKeys.QUEUE_RUNNERS` 集合中
    3. 一般会出现两种队列
        
        1. 第一个由tf.train.string_input_produecer函数创建，负责管理数据文件名
        2. 第二个由tf.train.shuffle_batch函数创建，负责管理样本（或者自定义一个队列实现，需要使用 `QueueRunner` 封装）
    
    4. 队列的类型
        
        1. `tf.FIFOQueue` 按入列顺序出列的队列
        
            `tf.FIFOQueue(capacity, dtypes, shapes=None, names=None, shared_name=None, name='fifo_queue')`
        
        2. `tf.RandomShuffleQueue` 随机顺序出列的队列
        
            `tf.RandomShuffleQueue(capacity=10, min_after_dequeue=2, dtypes="float")`
            
        3. `tf.PaddingFIFOQueue` 以固定长度批量出列的队列
        4. `tf.PriorityQueue` 带优先级出列的队列
        
    5. 队列的操作主要就是 : **出队**, **入队**
        
    6. 队列初始化
        
        `tf.FIFOQueue(capacity, dtypes, shapes=None, names=None, shared_name=None, name='fifo_queue')`
        
        1. `capacity` : 队列的容量
        2. `dtype` : 队列元素的数据类型
        3. `shapes` : 队列中的 `tensor` 的形状
      
    7. 队列阻塞
       
        1. 容量为0或者小于自低要求容量，还需要出队
        2. 容量已经满，还需要入队操作
    
2. `QueueRunner`
3. `Coordinator`